In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np
import pydeck as pdk
import seaborn as sns
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import importlib
sys.path.insert(0, os.path.abspath('../../'))

In [4]:
from osmnx_network_extract import CreateMcarptifFormat
from osmnx_network_extract import PrepareRequiredArcs
from osmnx_network_extract import PrepareGraph
from osmnx_network_extract import PrepareKeyLocations
from osmnx_network_extract import create_arc_id
from osmnx_network_extract import convert_to_int
from osmnx_network_extract import test_column_type
from osmnx_network_extract import test_column_exists

In [299]:
graph = {'u': list(range(5)), 'v': list(range(1, 6)), 'key': [0] * 5, 'travel_cost': [1] * 5, 'oneway': [True] * 5} 
graph = pd.DataFrame(graph)
graph

,u,v,key,travel_cost,oneway
0,0,1,0,1,True
1,1,2,0,1,True
2,2,3,0,1,True
3,3,4,0,1,True
4,4,5,0,1,True


In [300]:
graph_full = pd.concat([graph, graph, graph])
graph_full['travel_cost'] = list(range(1, graph_full.shape[0] + 1))
graph_full2 = graph_full.copy()
graph_full2['u'] = graph_full['v']
graph_full2['v'] = graph_full['u']
graph_full = pd.concat([graph_full, graph_full2])
graph_full['oneway'] = False
graph_full['key'] = list(range(graph_full.shape[0]))
graph_full = graph_full.reset_index(drop=True)

In [301]:
graph_full

,u,v,key,travel_cost,oneway
0,0,1,0,1,False
1,1,2,1,2,False
2,2,3,2,3,False
3,3,4,3,4,False
4,4,5,4,5,False
5,0,1,5,6,False
6,1,2,6,7,False
7,2,3,7,8,False
8,3,4,8,9,False
9,4,5,9,10,False


In [312]:
arcs = np.vstack((graph_full['u'].values, graph_full['v'].values)).T
G = nx.DiGraph()
G.add_edges_from(arcs)
nx.is_strongly_connected(G)

True

In [342]:
graph_full['oneway'] = True
prep = PrepareGraph(graph_full)
graph_full_prepared = prep.prep_osmnx_graph()

In [343]:
graph_full_prepared

,u,v,key,travel_cost,oneway,arc_id,arc_id_orig,u_orig,v_orig,arc_id_ordered,arc_id_ordered_orig,parallel
0,50,790,0,1,True,50-790-0,0-1-0,0.0,1.0,50-790-0,0-1-0,True
1,51,791,1,2,True,51-791-1,1-2-1,1.0,2.0,51-791-1,1-2-1,True
2,52,792,2,3,True,52-792-2,2-3-2,2.0,3.0,52-792-2,2-3-2,True
3,53,793,3,4,True,53-793-3,3-4-3,3.0,4.0,53-793-3,3-4-3,True
4,54,794,4,5,True,54-794-4,4-5-4,4.0,5.0,54-794-4,4-5-4,True
...,...,...,...,...,...,...,...,...,...,...,...,...
25,815,0,0,0,True,815-0-0,NaN,NaN,NaN,815-0-0,NaN,NaN
26,816,1,0,0,True,816-1-0,NaN,NaN,NaN,816-1-0,NaN,NaN
27,817,2,0,0,True,817-2-0,NaN,NaN,NaN,817-2-0,NaN,NaN
28,818,3,0,0,True,818-3-0,NaN,NaN,NaN,818-3-0,NaN,NaN


In [331]:
graph_full_prepared

,u,v,key,travel_cost,oneway,arc_id,arc_id_orig,u_orig,v_orig,arc_id_ordered,arc_id_ordered_orig,parallel
0,50,790,0,1,False,50-790-0,50-790-0,50,790,50-790,50-790,True
1,51,791,1,2,False,51-791-1,51-791-1,51,791,51-791,51-791,True
2,52,792,2,3,False,52-792-2,52-792-2,52,792,52-792,52-792,True
3,53,793,3,4,False,53-793-3,53-793-3,53,793,53-793,53-793,True
4,54,794,4,5,False,54-794-4,54-794-4,54,794,54-794,54-794,True
...,...,...,...,...,...,...,...,...,...,...,...,...
25,60,0,0,0,False,60-0-0,60-0-0,60,0,0-60,0-60,NaN
26,61,1,0,0,False,61-1-0,61-1-0,61,1,1-61,1-61,NaN
27,62,2,0,0,False,62-2-0,62-2-0,62,2,2-62,2-62,NaN
28,63,3,0,0,False,63-3-0,63-3-0,63,3,3-63,3-63,NaN
